<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase4/blob/main/Atividade4Corrigida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get update && apt-get install -y libgl1-mesa-glx
!pip install opencv-python matplotlib mediapipe

!pip install --upgrade numpy deepface


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,378 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [69.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,675 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,236 kB]
Hit:13 https://ppa.launchpadconte

In [3]:
import cv2
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from google.colab.patches import cv2_imshow
from deepface import DeepFace
import mediapipe as mp
import numpy as np
import gdown

In [4]:
# Baixar o vídeo diretamente do Google Drive
VIDEO_ID = "1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne"  # ID do arquivo no Google Drive
VIDEO_PATH = "/content/video.mp4"
gdown.download(f"https://drive.google.com/uc?id={VIDEO_ID}", VIDEO_PATH, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1B5PbZdUDi-r7Ac7BK3a3WdNppfQqM_Ne
To: /content/video.mp4
100%|██████████| 38.3M/38.3M [00:00<00:00, 58.6MB/s]


'/content/video.mp4'

In [ ]:
def detect_faces_and_activities(video_path, output_path, duration_seconds=None):
    """Detecta rostos, emoções e atividades no vídeo."""

    # Modelos do MediaPipe para detecção de atividades
    mp_holistic = mp.solutions.holistic
    holistic = mp_holistic.Holistic()

    # Modelo de detecção de rosto
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Abrindo o vídeo
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Erro: Não foi possível abrir o vídeo.")
        return

    # Obtendo propriedades do vídeo
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    # Definir limite de quadros para teste rápido
    if duration_seconds:
        max_frames = min(total_frames, int(fps * duration_seconds))
    else:
        max_frames = total_frames

    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    frame_count = 0  # Contador de quadros processados

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        frame_count += 1

        # Converter para escala de cinza (para detecção de rosto)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # Detectar rostos
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

        # Processar cada rosto detectado
        for (x, y, w, h) in faces:
            face = frame[y:y+h, x:x+w]  # Extrair a região do rosto

            try:
                # Analisar emoção usando DeepFace
                analysis = DeepFace.analyze(face, actions=['emotion'], enforce_detection=False)
                emotion = analysis[0]['dominant_emotion']  # Obter emoção dominante

                # Desenhar retângulo no rosto
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

                # Adicionar a emoção detectada no vídeo
                cv2.putText(frame, emotion, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

            except Exception as e:
                print(f"Erro na análise de emoção: {e}")

        # **Detecção de atividade humana (pose + mãos + rosto)**
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = holistic.process(frame_rgb)

        activity = "Desconhecida"

        if result.pose_landmarks:
            mp_drawing = mp.solutions.drawing_utils
            mp_drawing.draw_landmarks(frame, result.pose_landmarks, mp_holistic.POSE_CONNECTIONS)

            # Pegar marcos corporais
            landmarks = result.pose_landmarks.landmark
            left_shoulder = landmarks[mp_holistic.PoseLandmark.LEFT_SHOULDER].y
            left_knee = landmarks[mp_holistic.PoseLandmark.LEFT_KNEE].y

            # Determinar se a pessoa está em pé ou sentada
            if left_shoulder < left_knee:
                activity = "Em Pé"
            else:
                activity = "Sentado"

        if result.face_landmarks and result.right_hand_landmarks:
            # Detectar se está lendo: Cabeça inclinada + mãos próximas ao rosto
            nose_y = result.face_landmarks.landmark[1].y
            hand_y = result.right_hand_landmarks.landmark[8].y  # Ponta do dedo indicador

            if hand_y < nose_y:
                activity = "Lendo"

        elif result.face_landmarks:
            # Detectar se está falando: Boca aberta (diferença entre lábio superior e inferior)
            upper_lip = result.face_landmarks.landmark[13].y
            lower_lip = result.face_landmarks.landmark[14].y

            if abs(upper_lip - lower_lip) > 0.02:
                activity = "Falando"

        elif result.right_hand_landmarks:
            # Detectar se está gesticulando: Mãos levantadas e afastadas do rosto
            wrist_y = result.right_hand_landmarks.landmark[0].y  # Posição do pulso
            nose_y = result.face_landmarks.landmark[1].y if result.face_landmarks else 0.5

            if wrist_y < nose_y:
                activity = "Gesticulando"

        # Adicionar texto de atividade detectada no vídeo
        cv2.putText(frame, activity, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

        # Escrever o quadro no vídeo de saída
        out.write(frame)

        # Exibir o quadro no Google Colab
        cv2_imshow(frame)
        clear_output(wait=True)

    cap.release()
    out.release()
    cv2.destroyAllWindows()
    print(f"Vídeo processado salvo como {output_path}")


# **Exemplo de uso:**
detect_faces_and_activities(
    "/content/video.mp4",
    "/content/output_video.mp4",
    duration_seconds=9  # Teste rápido com apenas 10 segundos de vídeo
)


Vídeo processado salvo como /content/output_video.mp4
